# Machine Learning in Network Science
Group Challenge

***
by: Leonardo Basili, Paul Bédier, Lasse Schmidt

within: MS Data Sciences & Business Analytics

at: CentraleSupélec & ESSEC Business School
***

This notebook covers deep learning techniques, namely:
- Variational Graph Normalized Auto-Encoders (based on https://arxiv.org/abs/2108.08046) which allow us to learn graph embeddings in an unsupervised way (based on graph structure and node embeddings)

### 1. Import Packages

In [112]:
from importlib import reload
reload(analyseData)
reload(loadData)
reload(modeling)
reload(autoenc)

<module 'util.autoencoder' from '/Users/macbookpro/Documents/GitHub/Network-Science_Final-Project/util/autoencoder.py'>

In [86]:
# import own scripts
import util.analyse_Data as analyseData
#import util.preprocess_Data as prepData
import util.load_Data as loadData
import util.modeling as modeling
import util.autoencoder as autoenc

In [87]:
# parse & handle data
import os
import numpy as np
import pandas as pd

# modeling
import torch
from torch_geometric.nn import GAE, VGAE

# hyperparam optimization
from ray import tune, air

# evaluation
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report, confusion_matrix, ConfusionMatrixDisplay


# visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [88]:
# set matplotlib and seaborn settings for nicer plots
%matplotlib inline

SMALL_SIZE = 6
MEDIUM_SIZE = 8
BIGGER_SIZE = 10

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)    # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=MEDIUM_SIZE)   # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

### 2. Load Data for Modeling

In [113]:
# might take up to a minute
data, (G, G_train, G_trainval, node_info, train_tf, val_tf, trainval_tf, test_tf) = autoenc.load()

Number of positive edges for training: 3802
Number of positive edges for validation: 1085
Number of positive edges for test: 542
Number of edges in original graph: 5429
Number of edges in training graph: 3802
Number of non-existing edges generated: 29971
Number of negative edges for training: 3802
Number of negative edges for validation: 1085
Number of negative edges for test: 542
Enriching node features...


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:00<00:00, 13.30it/s]
/Users/macbookpro/Library/Python/3.9/lib/python/site-packages/networkx/algorithms/link_analysis/hits_alg.py:78: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A = nx.adjacency_matrix(G, nodelist=list(G), dtype=float)


Computing transition probabilities:   0%|          | 0/2708 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|██████████| 10/10 [00:00<00:00, 10.91it/s]


Create PyTorch Geometric dataset...


In [90]:
# where to save trial results to
ray_path = os.path.abspath("")+"/ray_results"
if not os.path.isdir(ray_path):
    os.mkdir(ray_path)

### 3. VGNAE Node Embeddings

https://github.com/SeongJinAhn/VGNAE/blob/main/main.py for Variational Graph Normalized Auto-Encoders

#### 3.1 Hyperparameter tuning

In [70]:
# tunable hyperparameter search space --> search using tune.choice([]), tune.uniform(lower, upper), tune.grid_search([])
config = {
    # log params in raytune
    "ray": True,
    
    # print results per epoch
    "verbose": False,
    
    # basic infos
    "data": data,
    "max_epochs": 50,
    "save": True, # if we want to save best model on validation set
    
    # model
    "model": "VGNAE",
    
    ## encoder
    "enc_channels": 64,
    "scaling": 1.8,
    "num_prop": tune.grid_search([4, 16, 32, 64, 128]),
    "teleport": 0, # tune.grid_search([0, 0.1, 0.2]),
    "dropout": 0, # tune.grid_search([0, 0.1, 0.2]),
    
    # optimizer
    "lr": tune.grid_search([1e-3, 1e-4, 1e-5]),
    "wd": 0,
}

In [71]:
# how many trials to run (if grid_search utilized, it will run this number per grid_search value)
num_samples = 1

# run experiment
result_grid = autoenc.run_ray_experiment(
    autoenc.train_validate, config, ray_path, num_samples,
    metric_columns = ["trn_auc", "val_auc", "max_val_auc", "training_iteration"],
    parameter_columns = ["scaling", "num_prop", "lr"]
)

2023-04-23 11:43:21,824	INFO worker.py:1553 -- Started a local Ray instance.
2023-04-23 11:48:36,496	INFO tune.py:798 -- Total run time: 313.82 seconds (304.79 seconds for the tuning loop).


#### 3.2 Result of Hyperparameter tuning

In [114]:
restored_tuner, result_grid = autoenc.open_validate_ray_experiment(
    "ray_results/train_validate_2023-04-23_11-43-18",
    autoenc.train_validate
)

2023-04-23 16:34:53,591	WARNING tuner_internal.py:292 -- The trainable will be overwritten - this should be done with caution: it's possible to supply an incompatible trainable, and there are no guarantees that the resumed experiment will continue successfully. If you encounter errors during training, ensure that you are passing in the same trainable that was passed into the initial `Tuner` object.


Loading results from ray_results/train_validate_2023-04-23_11-43-18...


2023-04-23 16:35:04,360	INFO experiment_analysis.py:789 -- No `self.trials`. Drawing logdirs from checkpoint file. This may result in some information that is out of sync, as checkpointing is periodic.


Done!

No errors! Number of terminated trials: 15


In [115]:
# get best score per trial (highest validation accuracy)
N = 10
best_result_df = result_grid.get_dataframe(
    filter_metric="val_auc", filter_mode="max"
)
best_result_df = best_result_df[["trial_id", "training_iteration", "config/enc_channels",
                                 "config/scaling", "config/num_prop", "config/lr", "config/wd", 
                                 "trn_loss", "val_loss", "trn_auc", "val_auc"]]
best_result_df = best_result_df.sort_values(by=["val_auc"], ascending = False)

if len(result_grid) > N:
    best_result_df = best_result_df.head(N)

best_result_df

,trial_id,training_iteration,config/enc_channels,config/scaling,config/num_prop,config/lr,config/wd,trn_loss,val_loss,trn_auc,val_auc
0,4982a_00000,42,64,1.8,4,0.00100,0,5.112204,1.409582,0.948185,0.825806
3,4982a_00003,41,64,1.8,16,0.00100,0,5.324944,1.422033,0.950026,0.823963
6,4982a_00006,47,64,1.8,32,0.00100,0,5.056614,1.652868,0.940163,0.819355
9,4982a_00009,48,64,1.8,64,0.00100,0,4.720571,1.399702,0.944503,0.818433
12,4982a_00012,48,64,1.8,128,0.00100,0,4.720572,1.399701,0.944503,0.818433
1,4982a_00001,1,64,1.8,4,0.00010,0,9.989462,6.556673,0.693056,0.661751
2,4982a_00002,1,64,1.8,4,0.00001,0,9.989462,6.556799,0.693056,0.661751
10,4982a_00010,35,64,1.8,64,0.00010,0,10.118744,7.350160,0.705155,0.660369
13,4982a_00013,35,64,1.8,128,0.00010,0,10.118744,7.350160,0.705155,0.660369
7,4982a_00007,35,64,1.8,32,0.00010,0,10.118716,7.350062,0.705024,0.659908


#### 3.3 Embeddings based on best model

In [102]:
# load best autoencoder
path = os.path.abspath("")+"/models/VGNAE_0.001_0_4982a_00000_autoencoder.pt"
#path = "models/autoencoder.pt"
model = VGAE(autoenc.Encoder(data.x.size()[1], 64, 1.5, 4, 0, 0))
model.load_state_dict(torch.load(path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [116]:
# get embeddings of nodes
embedding = autoenc.get_embeddings(model, data.x, data.train_pos_edges)

In [77]:
# plot embedding
plt.scatter(
    embedding[:, 0],
    embedding[:, 1])
plt.gca().set_aspect('equal', 'datalim')
plt.title('VGNAE projection (first 2 dim) of nodes')
plt.savefig('scatter_plot')
plt.close()

Embeddings look like a sphere, we cannot do any meaningful clusering based on this.

Let us now take our original VGNAE embeddings and put them into a pandas dataframe.

In [118]:
node_emb = pd.DataFrame(embedding).rename(columns = {val: f"x{val+1}" for val in range(embedding.shape[1])})

node_emb

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,...,x55,x56,x57,x58,x59,x60,x61,x62,x63,x64
0,-3.236286,-0.468748,0.200673,0.119956,-2.179250,0.696636,0.924463,-1.441486,0.961940,1.282785,...,1.646246,0.226134,-0.399453,1.185523,0.890281,1.064403,1.678228,0.644963,0.304780,0.644810
1,-0.689387,-0.065556,0.061150,-0.018767,-0.524612,0.274261,0.114785,-0.322952,0.256056,0.273089,...,0.293403,0.002241,-0.155225,0.475788,0.242563,0.012322,0.253677,0.065230,0.315171,-0.001243
2,-0.070377,-0.082472,-0.310569,-0.367790,-0.352006,0.236861,0.024659,0.453466,-0.217919,0.454189,...,0.173446,0.154276,0.043935,-0.186528,-0.616538,0.548091,-0.307670,-0.012849,-0.972306,-0.391044
3,-0.076936,0.014131,-0.440995,-0.316667,-0.161113,0.198380,-0.211003,0.361687,-0.219950,0.071757,...,0.268518,0.126637,0.083584,-0.035810,-0.756658,0.321382,-0.282552,0.023124,-0.949508,-0.349958
4,-0.073513,0.187677,-0.194551,-0.241421,-0.145095,0.182752,0.112181,0.249016,-0.242467,-0.010454,...,-0.155616,0.054306,0.116953,-0.239180,-0.127139,0.059047,-0.018299,-0.123850,-0.646065,-0.279567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2703,-0.588050,0.080777,0.228448,-0.312431,-0.262744,-0.159776,-0.168388,-0.133149,0.129800,-0.228947,...,0.184320,0.023790,0.180927,0.058787,-0.016759,-0.277593,0.178736,0.045534,0.190312,-0.196260
2704,-0.152371,0.163245,-0.113662,-0.145505,0.191914,0.372227,-0.054689,0.099280,0.411920,0.117131,...,0.224125,0.053014,-0.102753,-0.137425,0.137797,0.295350,-0.000475,0.202006,0.158019,-0.446322
2705,-0.108056,-0.093721,-0.021188,0.281823,-0.082897,0.069613,-0.124905,0.114227,-0.325545,-0.302511,...,0.232269,0.249294,-0.085221,-0.001192,-0.296016,0.222574,-0.007141,0.084271,-0.117485,-0.077285
2706,-0.083750,-0.094119,-0.211447,0.264650,0.013921,0.094120,-0.125289,0.194479,-0.373463,-0.138108,...,-0.132007,0.168180,-0.131623,-0.110672,-0.428557,0.014701,-0.196563,-0.094884,-0.062665,-0.361352


In [52]:
print(y_train_hat.columns)

Index(['sim', 'y', 'pred'], dtype='object')


#### 3.4 Compute edge features based on best model

In [119]:
# predict train
y_train_hat = pd.DataFrame(autoenc.get_similarity(model, data.x, data.train_pos_edges, data.train_edges))

y_train_hat = (y_train_hat
    .rename(columns = {0: "sim"})
    .assign(y = trainval_tf.loc[trainval_tf.train_mask == True].y.values)
    .assign(pred = lambda df_: (df_.sim > df_.sim.median()).astype(int))
)

print("ROC: ", roc_auc_score(y_train_hat.y, y_train_hat.sim))
print("Acc: ", accuracy_score(y_train_hat.y, y_train_hat.pred))

ROC:  0.9764215364930167
Acc:  0.948185165702262


In [120]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
ax.scatter(y_train_hat.loc[y_train_hat['y'] == 0, 'sim'], y_train_hat.loc[y_train_hat['y'] == 0, 'y'], label='0')
ax.scatter(y_train_hat.loc[y_train_hat['y'] == 1, 'sim'], y_train_hat.loc[y_train_hat['y'] == 1, 'y'], label='1')
ax.legend()
plt.xlabel('sim')
plt.ylabel('y')
plt.savefig('scatter_plot.png')
plt.close()

In [24]:
y_train_hat_clean = y_train_hat[["sim", "y"]].apply(pd.to_numeric, errors='coerce').dropna()
sns.pairplot(y_train_hat_clean[["sim", "y"]], hue = "y")

KeyError: "None of [Index(['sim', 'y'], dtype='object')] are in the [columns]"

In [121]:
# predict val
y_val_hat = pd.DataFrame(autoenc.get_similarity(model, data.x, data.train_pos_edges, data.val_edges))
print(len(y_val_hat))
y_val_hat = (y_val_hat
    .rename(columns = {0: "sim"})
    .assign(y = trainval_tf.loc[trainval_tf.val_mask == True].y.values)
    .assign(pred = lambda df_: (df_.sim > df_.sim.median()).astype(int))
)
print("ROC: ", roc_auc_score(y_val_hat.y, y_val_hat.sim))
print("Acc: ", accuracy_score(y_val_hat.y, y_val_hat.pred))

2170
ROC:  0.889936503217312
Acc:  0.8258064516129032


In [122]:
tmp = (val_tf
    .assign(sim  = y_val_hat.pred.values)
    .assign(dist = lambda df_: [np.linalg.norm(node_emb.loc[u].values-node_emb.loc[v].values) for u, v in zip(df_.source, df_.target)])
)

In [84]:
sns.pairplot(tmp[["sim", "dist", "y"]], hue = "y")

NameError: name 'tmp' is not defined

In [123]:
tmp[['target', 'source', 'y', 'sim', 'dist']].corr()

,target,source,y,sim,dist
target,1.000000,0.320768,-0.493566,-0.420214,-0.256462
source,0.320768,1.000000,-0.487168,-0.398406,-0.012116
y,-0.493566,-0.487168,1.000000,0.651613,0.011700
sim,-0.420214,-0.398406,0.651613,1.000000,-0.026005
dist,-0.256462,-0.012116,0.011700,-0.026005,1.000000


In [124]:
# predict test
y_test_hat = pd.DataFrame(autoenc.get_similarity(model, data.x, data.trainval_pos_edges, data.test_edges))
y_test_hat = (y_test_hat
    .rename(columns = {0: "sim"})
    .assign(pred = lambda df_: (df_.sim > df_.sim.median()).astype(int))
)
y_test_hat[["pred"]].value_counts()

pred
0       542
1       542
dtype: int64